# Bivariate calibration
***

**Autor:** Chus Casado<br>
**Date:** 11-06-2024<br>

**To do:**<br>
* [ ] Clean observed time series

**Questions:**<br>
* [x] What's the relation between likelihood and KGE? Answer: 
$$Likelihood = 1 - KGE$$
* [ ] Fit the $Q_{min}$ based on the inflow data. Hot to fit a GEV to minima.
* [ ] Analyse the optimized parameters. Try to find a pattern related to reservoir use, size, catchment area...

**Ideas:**<br>
* SpotPy can not apply any optimization algorithm to a bivariable calibration. Instead, it includes different sampling methods (not optimization) such as Latin Hypercube and Monte Carlo. These two methods are good to explore the parameter space, but will not provide the global optimal parameters. In this notebook I compute the Pareto front of the outflow and storage KGE, and select as the optimal parameter set the one that is closer to the (1, 1) point. That means that I give equal importance to outflow and storage, and that the best parameter set is a Euclidean distance from the optimum value of (1, 1). If that approach is good, I could use an optimazation algorithm like SCE-UA to find the global optimum, provided that the target metric is the composition of the KGE values for storage and outflow:
$$OF = \sqrt{{(1 - KGE_{outflow})^2 + (1 - KGE_{storage} )^2}}$$

In [1]:
import sys
sys.path.append('../../src/')

In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import spotpy
from spotpy.objectivefunctions import kge
import yaml
from pathlib import Path
from tqdm.notebook import tqdm
import glob
# import cartopy.crs as ccrs
# import cartopy.feature as cf

In [4]:
from lisfloodreservoirs.calibration.bivariate_lisflood import bivariate_6pars
from lisfloodreservoirs.reservoirs.lisflood import Lisflood
from lisfloodreservoirs.utils.metrics import KGEmod, pareto_front
from lisfloodreservoirs.utils.plots import plot_iterations

## Configuration

In [6]:
PATH_DATASETS = Path('Z:/nahaUsers/casadje/datasets/')

# results will be saved in this path
PATH_OUT = Path('../calibration/MC/6parameters/') #Path('../results/reservoirs/LISFLOOD/calibration/MC/6parameters/')
PATH_OUT.mkdir(parents=True, exist_ok=True)

### Calibration

In [7]:
# sequential mode
parallel = 'seq'

# SCEUA parameters
max_iter = 600
# complexes = 4
train_size = .7

## Data

### GloFAS

#### Reservoirs

In [9]:
# load shapefile of GloFAS reservoirs
reservoirs = gpd.read_file('../../GIS/reservoirs_analysis_US.shp')
reservoirs.set_index('ResID', drop=True, inplace=True)

print(f'{reservoirs.shape[0]} reservoirs in the shape file')

94 reservoirs in the shape file


#### Time series

In [10]:
# read GloFAS time series
path = Path('../../data/reservoirs/GloFAS/long_run')
glofas_ts = {}
for file in tqdm(glob.glob(f'{path}/*.csv')):
    id = int(file.split('\\')[-1].split('.')[0].lstrip('0'))
    if id not in reservoirs.index:
        continue
    glofas_ts[id] = pd.read_csv(file, parse_dates=True, dayfirst=False, index_col='date')
    
print(f'{len(glofas_ts)} reservoirs in the GloFAS time series')

# convert storage time series into volume
for id, df in glofas_ts.items():
    df.storage *= reservoirs.loc[id, 'CAP'] * 1e6

# period of GloFAS simulation
start, end = glofas_ts[id].first_valid_index(), glofas_ts[id].last_valid_index()

  0%|          | 0/143 [00:00<?, ?it/s]

94 reservoirs in the GloFAS time series


### ResOpsUS
#### Time series

In [11]:
path_ResOps = Path(PATH_DATASETS / 'reservoirs' / 'ResOpsUS')

resops_ts = {}
for glofas_id in tqdm(reservoirs.index):
    # load timeseries
    grand_id = reservoirs.loc[glofas_id, 'GRAND_ID']
    series_id = pd.read_csv(path_ResOps / 'time_series_all' / f'ResOpsUS_{grand_id}.csv', parse_dates=True, index_col='date')
    # remove empty time series
    series_id = series_id.loc[start:end]#.dropna(axis=1, how='all')
    # remove duplicated index
    series_id = series_id[~series_id.index.duplicated(keep='first')]
    # save in dictionary
    resops_ts[glofas_id] = series_id

print(f'{len(resops_ts)} reservoirs in the ResOpsUS time series')
    
# convert storage from hm3 to m3
for id, df in resops_ts.items():
    df.storage *= 1e6

  0%|          | 0/94 [00:00<?, ?it/s]

94 reservoirs in the ResOpsUS time series


## Markov chain sampling

In [ ]:
# GloFAS reservoir
for ResID in tqdm(reservoirs.index):
    
    # file where the calibration results will be saved
    dbname = f'{PATH_OUT}/{ResID:03}_samples'
    if os.path.isfile(dbname + '.csv'):
        print(f'The file {dbname}.csv already exists.')
        continue
            
    ## TIME SERIES
    try:
        # observed time series
        obs = resops_ts[ResID][['storage', 'inflow', 'outflow']].copy()
        obs[obs < 0] = np.nan

        # define calibration period
        if obs.outflow.isnull().all():
            print(f'Reservoir {ResID} is missing outflow records')
            # continue
        elif obs.storage.isnull().all():
            print(f'Reservoir {ResID} is missing storage records')
            # continue
        else:
            start_obs = max([obs[var].first_valid_index() for var in ['storage', 'outflow']])
            end_obs = min([obs[var].last_valid_index() for var in ['storage', 'outflow']])
            cal_days = timedelta(days=np.floor((end_obs - start_obs).days * train_size))
            start_cal = end_obs - cal_days

        # define train and test time series
        x_train = glofas_ts[ResID].inflow[start_cal:end_obs]
        y_train = obs.loc[start_cal:end_obs, ['storage', 'outflow']]
        x_test = glofas_ts[ResID].inflow[start:start_cal]
        y_test = obs.loc[start_obs:start_cal, ['storage', 'outflow']]
        
    except:
        print(f'ERROR. The time series of reservoir {ResID} could not be set up')
        continue
        
    ## SET UP SPOTPY
    try:
        # extract GloFAS reservoir parameters
        Vc, Vtot, Qmin = reservoirs.loc[ResID, ['clim', 'CAP', 'minq']]
        Vtot *= 1e6
        Vc *= Vtot

        # initialize the calibration setup of the LISFLOOD reservoir routine
        setup = bivariate_6pars(x_train, y_train.storage, y_train.outflow,
                                Vc, Vtot, Qmin,
                                KGEmod)

        # define the sampling method
        mc = spotpy.algorithms.mc(setup, parallel=parallel, dbname=dbname, dbformat='csv', save_sim=False)
    except:
        print(f'ERROR. The SpotPY set up of reservoir {ResID} could not be done')

    ## LAUNCH SAMPLING
    try:
        # start the sampler
        mc.sample(max_iter)
    except:
        print(f'ERROR. While sampling the reservoir {ResID}')
        continue
        
    ### ANALYSE RESULTS
    
    try:
        # read CSV of results
        results = pd.read_csv(f'{dbname}.csv')
        results.index.name = 'iteration'
        parcols = [col for col in results.columns if col.startswith('par')]
        
        # plot pairplot of the combined likelihood
        results['likelihood'] = results[['like1', 'like2']].pow(2).sum(axis=1).pow(.5)
        sns.pairplot(results, vars=parcols, corner=True, hue='likelihood', palette='Spectral_r', plot_kws={'s': 12})
        plt.savefig(PATH_OUT / f'{ResID:03}_pairplot.jpg', dpi=300, bbox_inches='tight');
        
    except:
        print(f'ERROR while reading results form reservoir {ResID}')
        continue
        
    try:
        # compute the Pareto front and the best iteration
        front = pareto_front(results.like1, results.like2)
        best_iter = front.pow(2).sum(axis=1).pow(.5).idxmin()
        plot_iterations(results, front, best_iter, save=PATH_OUT / f'{ResID:03}_pareto_front.jpg')

        # select optimal parameters
        best_iter = results.like1.idxmin()
        parvalues = {col[3:]: float(results.loc[best_iter, col]) for col in parcols}

        # export optimal parameters
        with open(f'{PATH_OUT}/{ResID:03}_optimal_parameters.yml', 'w') as file:
            yaml.dump(parvalues, file)
    except:
        print(f'ERROR while searching for optimal parameters in reservoir {ResID}')
        continue
        
    try:
        # # declare the reservoir with the optimal parameters
        # Vn, Vf = [parvalues[var] * Vtot for var in ['FFn', 'FFf']]
        # Vn_adj = Vn + parvalues['alpha'] * (Vf - Vn)
        # Qn, Qf = [setup.inflow.quantile(parvalues[var]) for var in ['QQn', 'QQf']]
        # k = parvalues['k']
        # res = Lisflood(Vc, Vn, Vn_adj, Vf, Vtot, Qmin, Qn, Qf)
        
        # declare the reservoir with the optimal parameters
        Vf = parvalues['FFf'] * Vtot
        Vn = parvalues['alpha'] * Vf
        Vn_adj = Vn + parvalues['beta'] * (Vf - Vn)
        Qf = setup.inflow.quantile(parvalues['QQf'])
        Qn = parvalues['gamma'] * Qf
        k = parvalues['k']
        res = Lisflood(Vc, Vn, Vn_adj, Vf, Vtot, Qmin, Qn, Qf)

        # define time series
        start_obs = max([resops_ts[ResID][var].first_valid_index() for var in ['storage', 'outflow']])
        x = glofas_ts[ResID][start_obs:]
        y = resops_ts[ResID][start_obs:]

        # simulate the whole period and analyse
        sim = res.simulate(x.inflow, y.storage[start_obs], k=k)
        res.scatter(sim, y, norm=False, title=ResID, save=PATH_OUT / f'{ResID:03}_scatter.jpg')
        res.lineplot({'GloFAS': x, 'cal': sim}, y, save=PATH_OUT / f'{ResID:03}_lineplot.jpg')

    except:
        print(f'ERROR while simulating with optimal parameters in reservoir {ResID}')

  0%|          | 0/94 [00:00<?, ?it/s]

Initializing the  Monte Carlo (MC) sampler  with  600  repetitions
Starting the MC algorithm with 600 repetitions...
Initialize database...
['csv', 'hdf5', 'ram', 'sql', 'custom', 'noData']
* Database file '..\calibration\MC\6parameters/007_samples.csv' created.
4 of 600, min objf=0.735107, max objf=0.81074, time remaining: 00:05:06
8 of 600, min objf=0.732252, max objf=0.845545, time remaining: 00:05:39
12 of 600, min objf=0.732252, max objf=1.01033, time remaining: 00:05:47
16 of 600, min objf=0.732252, max objf=1.01418, time remaining: 00:05:55
19 of 600, min objf=0.731455, max objf=1.01418, time remaining: 00:05:59
23 of 600, min objf=0.731455, max objf=1.01418, time remaining: 00:05:58
27 of 600, min objf=0.731455, max objf=1.05295, time remaining: 00:05:57
31 of 600, min objf=0.731455, max objf=1.05295, time remaining: 00:05:56
34 of 600, min objf=0.731455, max objf=1.05295, time remaining: 00:05:57
38 of 600, min objf=0.731455, max objf=1.05295, time remaining: 00:05:56
42 of 60

## Analyse results

In [ ]:
for ResID in [12]:#tqdm(reservoirs.index):
    
    try:

In [ ]:
ResID = 481 #507

In [ ]:
        # file where the calibration results will be saved
        dbname = f'{PATH_OUT}/{ResID:03}_samples'

        # read CSV of results
        results = pd.read_csv(f'{dbname}.csv')
        results.index.name = 'iteration'
        parcols = [col for col in results.columns if col.startswith('par')]

        # compute the Pareto front and the best iteration
        front = pareto_front(results.like1, results.like2)
        best_iter = front.pow(2).sum(axis=1).pow(.5).idxmin()

In [ ]:
        # select and export optimal parameters
        parvalues = {col[3:]: float(results.loc[best_iter, col]) for col in parcols}
        with open(f'{PATH_OUT}/{ResID:03}_optimal_parameters.yml', 'w') as file:
            yaml.dump(parvalues, file)

In [ ]:
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.scatter(results.like1, results.like2, s=1, c='gray', label='iteration', alpha=.2)
    ax.scatter(*front.loc[best_iter, ['like1', 'like2']], c='steelblue', label='optimum', s=4)
    ax.plot(front.like1, front.like2, c='k', lw=1, ls=':', label='pareto front', zorder=0)
    # vmax = np.ceil(front[['like1', 'like2']].max().max() / .2) * .2 + .025
    vmax = None
    # # ticks = np.arange(0, vmax, .25)
    ax.set(xlim=(-.025, vmax),
           xlabel=r'$L_{outflow}$',
           # xticks=ticks,
           ylim=(-.025, vmax),
           ylabel=r'$L_{storage}$',
           # yticks=ax.get_xticks(),
          )
    ax.set_title(ResID)
    fig.legend(frameon=False, loc=1, bbox_to_anchor=[1.175, .7, .1, .2]);
    plt.savefig(PATH_OUT / f'{ResID:03}_pareto_front.jpg', dpi=300, bbox_inches='tight');

In [ ]:
# observed time series
obs = resops_ts[ResID][['storage', 'inflow', 'outflow']].copy()
obs[obs < 0] = np.nan
        
# define train and test time series
start_obs = max([obs[var].first_valid_index() for var in ['storage', 'outflow']])
end_obs = min([obs[var].last_valid_index() for var in ['storage', 'outflow']])
cal_days = timedelta(days=np.floor((end_obs - start_obs).days * train_size))
start_cal = end_obs - cal_days

# define train and test time series
x_train = glofas_ts[ResID].inflow[start_cal:end_obs]
y_train = obs.loc[start_cal:end_obs, ['storage', 'outflow']]
x_test = glofas_ts[ResID].inflow[start:start_cal]
y_test = obs.loc[start:start_cal, ['storage', 'outflow']]

In [ ]:
# extract GloFAS reservoir parameters
Vc, Vtot, Qmin = reservoirs.loc[ResID, ['clim', 'CAP', 'minq']]
Vtot *= 1e6
Vc *= Vtot

# initialize the calibration setup of the LISFLOOD reservoir routine
setup = bivariate_6pars(x_train, y_train.storage, y_train.outflow,
                        Vc, Vtot, Qmin,
                        KGEmod)

In [ ]:
results[['KGE1', 'KGE2']] = np.nan
# idx = results[results.like2.isnull()].index[1]
for idx in tqdm(results.index):
    
    parvalues = {col[3:]: float(results.loc[idx, col]) for col in parcols}

    # declare the reservoir with the optimal parameters
    Vn, Vf = [parvalues[var] * Vtot for var in ['FFn', 'FFf']]
    Vn_adj = Vn + parvalues['alpha'] * (Vf - Vn)
    Qn, Qf = [setup.inflow.quantile(parvalues[var]) for var in ['QQn', 'QQf']]
    k = parvalues['k']
    res = Lisflood(Vc, Vn, Vn_adj, Vf, Vtot, Qmin, Qn, Qf)

    # simulate the whole period and analyse
    sim = res.simulate(x_train, y_train.storage[start_cal], k=k)

    results.loc[idx, 'KGE1'] = 1 - KGEmod(y_train.outflow, sim.outflow)[0]
    results.loc[idx, 'KGE2'] = 1 - KGEmod(y_train.storage, sim.storage)[0]

In [ ]:
results[['KGE1', 'KGE2']].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(results.KGE1, results.KGE2, s=1, c='gray', label='iteration', alpha=.2)
# vmax = np.ceil(front[['like1', 'like2']].max().max() / .2) * .2 + .025
# vmax = None
# ticks = np.arange(0, vmax, .25)
ax.set(xlim=(-.025, None),
       xlabel=r'$KGE_{outflow}$',
       # xticks=ticks,
       ylim=(-.025, None),
       ylabel=r'$KGE_{storage}$',
       # yticks=ax.get_xticks(),
      )
ax.set_title(ResID)
fig.legend(frameon=False, loc=1, bbox_to_anchor=[1.175, .7, .1, .2]);

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 4))

for i, (ax, var) in enumerate(zip(axes, ['outflow', 'storage']), start=1):
    ax.scatter(results[f'KGE{i}'], results[f'like{i}'], s=1, c='gray', label='iteration', alpha=.2)
    ax.set(title=var,
           xlabel='KGE',
           # xlim=(None, 1.02),
           # ylim=(-.02, None),
          )
    if ax == axes[0]:
        ax.set_ylabel('likelihood')
    
# axes[0].axhline(1, ls=':', c='k', lw=.5)
# axes[0].axvline(0, ls=':', c='k', lw=.5);

In [ ]:
# compute the Pareto front and the best iteration
front = pareto_front(1 - results.KGE1, 1 - results.KGE2)
front['d'] = (1 - front[['KGE1', 'KGE2']]).pow(2).sum(axis=1).pow(.5)
best_iter = front.d.idxmin()

In [ ]:
front

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(1 - results.KGE1, 1 - results.KGE2, s=1, c='gray', label='iteration', alpha=.2)
ax.scatter(*front.loc[best_iter, ['KGE1', 'KGE2']], c='steelblue', label='optimum', s=4)
ax.scatter(*front.loc[best_iter, ['KGE1', 'KGE2']], c='steelblue', label='optimum', s=4)
# ax.plot(front.KGE1, front.KGE2, c='k', lw=1, ls=':', label='pareto front', zorder=0)
# vmax = np.ceil(front[['KGE1', 'KGE2']].max().max() / .2) * .2 + .025
# vmax = None
# # ticks = np.arange(0, vmax, .25)
ax.set(xlim=(-.025, None),
       xlabel=r'$L_{outflow}$',
       # xticks=ticks,
       ylim=(-.025, None),
       ylabel=r'$L_{storage}$',
       # yticks=ax.get_xticks(),
      )
ax.set_title(ResID)
fig.legend(frameon=False, loc=1, bbox_to_anchor=[1.175, .7, .1, .2]);
plt.savefig(PATH_OUT / f'{ResID:03}_pareto_front.jpg', dpi=300, bbox_inches='tight');

In [ ]:
results.loc[results.like2.isnull(), parcols].iloc[0]

In [ ]:
        # select and export optimal parameters
        parvalues = {col[3:]: float(results.loc[best_iter, col]) for col in parcols}
        with open(f'{PATH_OUT}/{ResID:03}_optimal_parameters.yml', 'w') as file:
            yaml.dump(parvalues, file)

        # declare the reservoir with the optimal parameters
        Vn, Vf = [parvalues[var] * Vtot for var in ['FFn', 'FFf']]
        Vn_adj = Vn + parvalues['alpha'] * (Vf - Vn)
        Qn, Qf = [setup.inflow.quantile(parvalues[var]) for var in ['QQn', 'QQf']]
        k = parvalues['k']
        res = Lisflood(Vc, Vn, Vn_adj, Vf, Vtot, Qmin, Qn, Qf)

        # define time series
        start_obs = max([resops_ts[ResID][var].first_valid_index() for var in ['storage', 'outflow']])
        x = glofas_ts[ResID][start_obs:]
        y = resops_ts[ResID][start_obs:]

        # simulate the whole period and analyse
        sim = res.simulate(x.inflow, y.storage[start_obs], k=k)
        res.scatter(sim, y, norm=False, title=ResID, save=PATH_OUT / f'{ResID:03}_scatter.jpg')
        res.lineplot({'GloFAS': x, 'cal': sim}, y, save=PATH_OUT / f'{ResID:03}_lineplot.jpg')

    except:
        print(f'ERROR. {ResID}')
        continue

***

In [ ]:
    # extract GloFAS reservoir parameters
    Vc, Vtot, Qmin = reservoirs.loc[ResID, ['clim', 'CAP', 'minq']]
    Vtot *= 1e6
    Vc *= Vtot

    # initialize the calibration setup of the LISFLOOD reservoir routine
    setup = bivariate_6pars(x, y.storage, y.outflow,
                            Vc, Vtot, Qmin,
                            KGEmod)

In [ ]:
    simulations = {}
    front[['KGE1', 'KGE2']] = np.nan
    for i in tqdm(front.index):
        Q, S = setup.simulation(results.loc[i, parcols])
        simulations[i] = {'outflow': Q, 'storage': S}
        front.loc[i, 'KGE1'] = KGEmod(y.outflow, Q)[0]
        front.loc[i, 'KGE2'] = KGEmod(y.storage, S)[0]

In [ ]:
    # plot simulations in the pareto front
    fig, axes = plt.subplots(nrows=2, figsize=(12, 6), sharex=True)
    for i, (ax, var, unit) in enumerate(zip(axes, ['outflow', 'storage'], ['m3/s', 'm3']), start=1):
        ax.plot(y[var], c='k', lw=.5, label='obs', zorder=2)
        ax.plot(simulations[best_iter][var], c='steelblue', lw=1, label='best iteration', zorder=3)
        for s, dct in simulations.items():
            ax.plot(dct[var], c='gray', lw=.5, alpha=.2, label='pareto front')
        ax.set(title='{0} KGE={1:.2f}'.format(var, front.loc[best_iter, f'KGE{i}']),
               ylabel=unit)
    ax.set(xlim=(start_cal, end))
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles[:3], labels[:3], frameon=False, loc=8, ncol=3);